# 迷路を走るランダムエージェント
参考文献：小川雄太郎著「つくりながら学ぶ！深層強化学習」<br>
Tomoko Ozeki

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## 環境：迷路を作る

In [ ]:
# 図を描く大きさと図の変数名を宣言
fig = plt.figure(figsize=((5,5)))
ax = plt.gca()

# 赤い壁を描く
plt.plot([1,1], [0,1], color='red', linewidth=2)
plt.plot([1,2], [2,2], color='red', linewidth=2)
plt.plot([2,2], [2,1], color='red', linewidth=2)
plt.plot([2,3], [1,1], color='red', linewidth=2)

# 状態を示す文字S0~S8を描く
plt.text(0.5, 2.5, 'S0', size=14, ha='center')
plt.text(1.5, 2.5, 'S1', size=14, ha='center')
plt.text(2.5, 2.5, 'S2', size=14, ha='center')
plt.text(0.5, 1.5, 'S3', size=14, ha='center')
plt.text(1.5, 1.5, 'S4', size=14, ha='center')
plt.text(2.5, 1.5, 'S5', size=14, ha='center')
plt.text(0.5, 0.5, 'S6', size=14, ha='center')
plt.text(1.5, 0.5, 'S7', size=14, ha='center')
plt.text(2.5, 0.5, 'S8', size=14, ha='center')
plt.text(0.5, 2.3, 'START', ha='center')
plt.text(2.5, 0.3, 'GOAL', ha='center')

# 描画範囲の設定と目盛りを消す設定
ax.set_xlim(0, 3)
ax.set_ylim(0, 3)
plt.tick_params(axis='both', which='both', bottom='off', top='off',
               labelbottom='off', right='off', left='off', labelleft='off')

# 現在地S0に緑丸を描画する
line, = ax.plot([0.5], [2.5], marker="o", color='c', markersize=60)
plt.show()

## エージェントを作る

### パラメータの初期値
行は状態 0~7，列は行動　up, right, down, left

In [ ]:
theta_0 = np.array([[np.nan, 1, 1, np.nan],     #S0
                   [np.nan, 1, np.nan, 1],      #S1
                   [np.nan, np.nan, 1, 1],      #S2
                   [1, 1, 1, np.nan],           #S3
                   [np.nan, np.nan, 1, 1],      #S4
                   [1, np.nan, np.nan, np.nan], #S5
                   [1, np.nan, np.nan, np.nan], #S6 
                   [1, 1, np.nan, np.nan]
                   ])

### パラメータを方策に変換する関数

In [ ]:
def theta2pi(theta):
    '''単純に割合を計算する'''
    
    [m, n] = theta.shape # 行列のサイズを取得
    pi = np.zeros((m, n))
    for i in range(0, m):
        pi[i, :] = theta[i, :] / np.nansum(theta[i, :])
    
    pi = np.nan_to_num(pi) # nanを0に変換する
    
    return pi

### パラメータを方策に変換

In [ ]:
pi_0 = theta2pi(theta_0)
print(pi_0)

### 行動を選択し、次の状態へ移る

In [ ]:
def get_next_s(pi, s):
    direction = ["up", "right", "down", "left"]
    
    next_direction = np.random.choice(direction, p=pi[s, :])
    # pi[s, :] の確率にしたがって，directionを選択する
    
    if next_direction == "up":
        s_next = s - 3
    elif next_direction == "right":
        s_next = s + 1
    elif next_direction == "down":
        s_next = s + 3
    else:
        s_next = s - 1
    
    return s_next

### 迷路内をエージェントがゴールするまで移動する関数

In [ ]:
def goal_maze(pi):
    s = 0
    state_history = [0] # エージェントの移動を記録するリスト
    
    while(1): # ゴールするまで繰り返す
        next_s = get_next_s(pi, s)
        state_history.append(next_s) # 記録リストに次の状態を追加
        
        if next_s == 8:  # ゴールならエピソード終了
            break
        else:
            s = next_s
        
    return state_history

### ゴールをめざして進む

In [ ]:
state_history = goal_maze(pi_0)

print(state_history)
print("迷路を解くのにかかったステップ数は" + str(len(state_history) - 1) + "です")

### アニメーションを作る
参考URL http://louistiao.me/posts/notebooks/embedding-matplotlib-animations-in-jupyter-as-interactive-javascript-widgets/

In [ ]:
from matplotlib import animation
from IPython.display import HTML

In [ ]:
def init():
    '''背景画像の初期化'''
    line.set_data([], [])
    return (line,)

def animate(i):
    '''フレームごとの描画内容'''
    state = state_history[i] # 現在の場所を描く
    x = (state % 3) + 0.5    # 状態のx座標
    y = 2.5 - int(state / 3) # 状態のy座標
    line.set_data(x, y)
    return (line)

# 初期化関数とフレームごとの描画関数を用いて動画を作成する
anim = animation.FuncAnimation(fig, animate, init_func=init, frames=len(state_history), interval=200, repeat=False)

HTML(anim.to_jshtml())